### --- Day 14: Restroom Redoubt ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/14

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 14);
WriteLines(inputLines);

Loading puzzle file: Day14.txt
Total lines: 500
Max line length: 18

p=39,34 v=40,73
p=43,71 v=-56,-11
p=87,9 v=71,41
p=67,40 v=12,6
p=54,54 v=-22,73


In [4]:
string[] testInputLines = [
    "p=0,4 v=3,-3",
    "p=6,3 v=-1,-3",
    "p=10,3 v=-1,2",
    "p=2,0 v=2,-1",
    "p=0,0 v=1,3",
    "p=3,0 v=-2,-2",
    "p=7,6 v=-1,-3",
    "p=3,0 v=-1,-2",
    "p=9,3 v=2,3",
    "p=7,3 v=-1,2",
    "p=2,4 v=2,-3",
    "p=9,5 v=-3,-3",
];

This first part looks fairly straightforward: move by `100 x velocity`, calculate the position modulo room size.

In [5]:
record struct Robot(Point Position, Point Velocity) {}

In [6]:
Robot ParseRobot(string inputLine)
{
    var bits = inputLine.Split(["p=", " v=", ","], StringSplitOptions.RemoveEmptyEntries)
                .Select(int.Parse)
                .ToArray();
    
    var (pX, pY, vX, vY) = bits;
    return new((pX, pY), (vX, vY));
}

In [7]:
// The robots outside the actual bathroom are in a space which is 101 tiles wide
// and 103 tiles tall (when viewed from above). However, in this example, the
// robots are in a space which is only 11 tiles wide and 7 tiles tall.

using Room = (int Width, int Height);

Room testRoom = new(11, 7);
Room part1Room = new(101, 103);

In [8]:
Point Walk(Robot robot, Room room, int seconds)
{
    var newX = WalkOne(robot.Position.X, robot.Velocity.X, seconds, room.Width);
    var newY = WalkOne(robot.Position.Y, robot.Velocity.Y, seconds, room.Height);

    return (newX, newY);
}

int WalkOne(int position, int velocity, int seconds, int size)
{
    var newPos = (position + velocity * seconds) % size;

    newPos = newPos switch {
        < 0 => newPos + size,
        _ => newPos
    };

    return newPos;
}

int GetQuadrant(Point point, Room room) 
{
    var xHalf = room.Width / 2;
    int yHalf = room.Height / 2;

    // Quadrants are 1, 2, 3, 4, clockwise from top-left. 0 means on the border
    // therefore no quadrant
    var quadrant = (point.X - xHalf, point.Y - yHalf) switch {
        (< 0, < 0) => 1,
        (> 0, < 0) => 2,
        (> 0, > 0) => 3,
        (< 0, > 0) => 4,
        _ => 0
    };

    return quadrant;
}

int WalkAll(string[] inputLines, Room room)
{
    const int seconds = 100;

    var x = from inputLine in inputLines
            let robot = ParseRobot(inputLine)
            let newPos = Walk(robot, room, seconds)
            group newPos by GetQuadrant(newPos, room) into quadrants
            select (quadrant: quadrants.Key, count: quadrants.Count());

    return x.Where(qc => qc.quadrant != 0).Select(qc => qc.count).Aggregate(1, (a, b) => a * b);
}

In [9]:
// In this example, the quadrants contain 1, 3, 4, and 1 robot. Multiplying
//these together gives a total safety factor of 12.

var testAnswer = WalkAll(testInputLines, testRoom);
Console.WriteLine(testAnswer);

12


In [10]:
// Predict the motion of the robots in your list within a space which is 101
// tiles wide and 103 tiles tall. What will the safety factor be after exactly 100
// seconds have elapsed?

var part1Answer = WalkAll(inputLines, part1Room);
Console.WriteLine(part1Answer);

211773366


In [11]:
// 211773366 is correct!
Ensure(211773366, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/14

Ok, that is a strange one! A Christmas tree could be many different shapes: single triangle, jagged, etc. There could be a base, decorations, it could be filled or an outline. Hmmm!

I'm guessing I'll just have to iterate the movements until a christmas tree appears. This is one time where a notebook might be a disadvantage, as if this were a console app I could animate the screen for each second of movement. Instead, I guess I'll write to a file and look at that.

In [13]:
// True if the current robo-state might be in a tree configuration. Allows us to
// test different theories.
delegate bool MightBeTreeFunc(Robot[] robots, Room room, int seconds);

In [14]:
void SearchForTrees(string[] inputLines, Room room, MightBeTreeFunc mightBeTree, int treesToFind, string filename)
{
    var robots = inputLines.Select(ParseRobot).ToArray();

    SafetyLimit sl = new();
    using var writer = new StreamWriter(filename, append: false);
    int seconds = 0;
    int timesFound = 0;
    while (true)
    {
        sl.EnsureBelow(1_000_000);

        if (mightBeTree(robots, room, seconds))
        {
            timesFound++;
            writer.WriteLine($"Found potential tree after {seconds} seconds");
            
            Console.Write($"{seconds}, ");
            if (timesFound % 10 == 0) { Console.WriteLine(); }

            Render(robots, room, writer);

            if (timesFound == treesToFind)
            {
                Console.WriteLine();
                Console.WriteLine($"Stopping after {timesFound} trees found in {seconds} seconds");
                return; 
            }
        }

        // Walk for one second
        foreach (var (i, robot) in robots.Index())
        {
            robots[i] = robot with { Position = Walk(robot, room, 1) };
        }
        
        seconds++;
    }
}

void Render(Robot[] robots, Room room, TextWriter writer)
{
    var rows = room.Height;
    var cols = room.Width;

    foreach (var row in Enumerable.Range(0, rows))
    {
        StringBuilder sb = new(cols);
        foreach (var col in Enumerable.Range(0, cols))
        {
            var tile = robots.Any(r => r.Position == (col, row)) switch {
                true => '#',
                _ => '.'
            };
            sb.Append(tile);
        }
        writer.WriteLine(sb);
    }
}

As a first attempt, let's assume the christmas tree is an outline that is triangular at the top, with the topmost point in the middle of the top row. We can render all occurrences of this and see if it looks like a tree.

In [15]:
bool IsTopMiddle(Robot[] robots, Room room, int seconds) => robots.Any(r => r.Position == (room.Width / 2, 0));

SearchForTrees(inputLines, part1Room, IsTopMiddle, 50, "IsTopMiddle.txt");

26, 44, 45, 55, 60, 65, 71, 93, 103, 126, 
132, 135, 141, 173, 176, 204, 218, 234, 253, 265, 
271, 302, 335, 337, 351, 356, 401, 421, 456, 466, 
467, 479, 517, 573, 575, 664, 671, 693, 702, 722, 
740, 766, 777, 790, 792, 862, 876, 880, 936, 939, 

Stopping after 50 trees found in 939 seconds


Hmm, not much happening there, basically white noise. Maybe we need to specify more of the triangle

In [16]:
bool IsMoreTriangle(Robot[] robots, Room room, int seconds)
{
    Point topMiddle = (room.Width / 2, 0);
    Point triLeft = topMiddle + Down + Left;
    Point triRight = topMiddle + Down + Right;

    Point[] allChecks = [topMiddle, triLeft, triRight];

    return allChecks.All(checkPoint => robots.Any(robot => robot.Position == checkPoint));
}

SearchForTrees(inputLines, part1Room, IsMoreTriangle, 50, "IsMoreTriangle.txt");

421, 2144, 10824, 12547, 21227, 22950, 31630, 33353, 42033, 43756, 
52436, 54159, 62839, 64562, 73242, 74965, 83645, 85368, 94048, 95771, 
104451, 106174, 114854, 116577, 125257, 126980, 135660, 137383, 146063, 147786, 
156466, 158189, 166869, 168592, 177272, 178995, 187675, 189398, 198078, 199801, 
208481, 210204, 218884, 220607, 229287, 231010, 239690, 241413, 250093, 251816, 

Stopping after 50 trees found in 251816 seconds


Bummer, looks about the same!

Let's try a new theory: whatever the shape of the tree, it is likely symmetrical in shape down the middle, so let's find instances where the positions are roughly symmetrical, i.e., the same number of robots in the left / right quadrants.

In [17]:
bool IsSymmetrical(Robot[] robots, Room room, int seconds)
{
    var quadrants = robots.GroupBy(r => GetQuadrant(r.Position, room)).OrderBy(q => q.Key).Select(qg => (decimal)qg.Count()).ToArray();

    var (q0, q1, q2, q3) = quadrants;

    return (q1 / q2) switch {
        >= 0.95m and <= 1.05m => true,
        _ => false
    };
}

SearchForTrees(inputLines, part1Room, IsSymmetrical, 50, "IsSymmetrical.txt");

3, 7, 8, 9, 10, 13, 14, 15, 17, 18, 
19, 20, 25, 27, 28, 31, 33, 34, 38, 40, 
41, 42, 51, 52, 54, 67, 68, 77, 84, 87, 
88, 89, 91, 92, 96, 98, 101, 104, 105, 106, 
108, 110, 112, 119, 122, 125, 126, 128, 130, 132, 

Stopping after 50 trees found in 132 seconds


Still looks like noise. I realise the error of my ways now: the noise is roughly balanced between the two quadrants, so this heuristic would never work.

We could try and get clever with some kind of row-wise symmetry heuristic, but at this point, let's just render a sh*tload of trees and look for clues...

In [18]:
bool ShowEverything(Robot[] robots, Room room, int seconds) => true;

SearchForTrees(inputLines, part1Room, ShowEverything, 300, "allTrees.txt");

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 
20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 
60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 
70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 
80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 
90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 
100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 
110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 
120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 
130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 
140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 
150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 
160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 
170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 
180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 
190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 
200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 
210, 211, 212, 213, 214, 215, 216, 217,

Hmm, most of the renders just look like noise, but interestingly after 72 seconds the robots appear to "organise" slightly. This pattern occurs again every 101 seconds thereafter, ie 173, 274, etc.

Let's render a bunch of these ones and see what happens...

In [19]:
bool IsEvery101(Robot[] robots, Room room, int seconds) => seconds % 101 == 72;

SearchForTrees(inputLines, part1Room, IsEvery101, 100, "IsEvery101.txt");

72, 173, 274, 375, 476, 577, 678, 779, 880, 981, 
1082, 1183, 1284, 1385, 1486, 1587, 1688, 1789, 1890, 1991, 
2092, 2193, 2294, 2395, 2496, 2597, 2698, 2799, 2900, 3001, 
3102, 3203, 3304, 3405, 3506, 3607, 3708, 3809, 3910, 4011, 
4112, 4213, 4314, 4415, 4516, 4617, 4718, 4819, 4920, 5021, 
5122, 5223, 5324, 5425, 5526, 5627, 5728, 5829, 5930, 6031, 
6132, 6233, 6334, 6435, 6536, 6637, 6738, 6839, 6940, 7041, 
7142, 7243, 7344, 7445, 7546, 7647, 7748, 7849, 7950, 8051, 
8152, 8253, 8354, 8455, 8556, 8657, 8758, 8859, 8960, 9061, 
9162, 9263, 9364, 9465, 9566, 9667, 9768, 9869, 9970, 10071, 

Stopping after 100 trees found in 10071 seconds


Aha! There it is! After 7344 seconds. It is symmetrical as suspected, and trianglular at the top, but not located in the middle of the room! It's also a filled tree, not an outline.

In [20]:
// What is the fewest number of seconds that must elapse for the robots to
// display the Easter egg?

var part2Answer = 7344;

In [21]:
// 7344 is correct!
Ensure(7344, part2Answer);

Just out of interest, I took a look at [Peter Norvig's solution](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2024.ipynb) to see if there's an "algorithmic" way to solve this. 

After a few tree-hunting expeditions of his own, Peter eventually used an algorithm to find when the robots are most tightly coalesced around a centroid. Looks like that worked, and it looks like fun, so let's try it!

In [22]:
IEnumerable<(int distance, int seconds)> WalkDistances(string[] inputLines, Room room, int maxSeconds)
{
    var robots = inputLines.Select(ParseRobot).ToArray();

    var seconds = 0;
    while (seconds <= maxSeconds)
    {
        var distance = DistanceFromCentroid(robots, room);
        
        yield return (distance, seconds);

        // Walk for one second
        foreach (var (i, robot) in robots.Index())
        {
            robots[i] = robot with { Position = Walk(robot, room, 1) };
        }

        seconds++;
    }
}

int DistanceFromCentroid(Robot[] robots, Room room)
{
    var meanX = robots.Select(r => r.Position.X).Sum() / robots.Length;
    var meanY = robots.Select(r => r.Position.Y).Sum() / robots.Length;

    Point centroid = (meanX, meanY);

    var meanDistance = robots.Select(r => SimpleDistance(r.Position, centroid)).Sum() / robots.Length;

    return meanDistance;
}

int SimpleDistance(Point a, Point b)
{
    var diff = b - a;
    return Math.Abs(diff.X) + Math.Abs(diff.Y);
}

In [23]:
foreach (var (distance, seconds) in WalkDistances(inputLines, part1Room, 10_000).OrderBy(wd => wd.distance).Take(10))
{
    Console.WriteLine($"Distance after {seconds} seconds is {distance}");
}

Distance after 7344 seconds is 25
Distance after 2297 seconds is 35
Distance after 134 seconds is 36
Distance after 340 seconds is 36
Distance after 1061 seconds is 36
Distance after 3327 seconds is 36
Distance after 4460 seconds is 36
Distance after 5078 seconds is 36
Distance after 8477 seconds is 36
Distance after 8786 seconds is 36


Sure enough, there's our tree at the top with the lowest mean distance.